# Multivariate Time Series Forecasting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
series = pd.read_table('household_power_consumption.csv', sep=',')
series

,datetime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...,...
86395,2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
86396,2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
86397,2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0
86398,2007-02-14 17:22:00,0.524,0.000,241.08,2.2,0.0,0.0,0.0


In [3]:
series_ = series.values[:,1:]

series_min = np.min(series_, axis=0)
series_max = np.max(series_, axis=0)

In [4]:
split_time = 69120

t_train = np.array(range(0,split_time+1))
x_train = (series_[:split_time,:]-series_min)/series_max

t_test = np.array(range(split_time+1,series.shape[0]+1))
x_test = (series_[split_time:,:]-series_min)/series_max

In [5]:
n_past = 24  
n_future = 24 
window_size = n_past + n_future

X_train = []
Y_train = []
for i in range(x_train.shape[0]-window_size):
    X_train.append(x_train[i:i+n_past,:])
    Y_train.append(x_train[i+n_past:i+window_size,:])
X_train = np.array(X_train, dtype='float32')
Y_train = np.array(Y_train, dtype='float32')

X_test = []
Y_test = []
for i in range(x_test.shape[0]-window_size):
    X_test.append(x_test[i:i+n_past,:])
    Y_test.append(x_test[i+n_past:i+window_size,:])
X_test = np.array(X_test, dtype='float32')
Y_test = np.array(Y_test, dtype='float32')

In [6]:
def Model(params):
    dims = params['dims']
    X_train = params['X_train']
    Y_train = params['Y_train']
    epochs = params['epochs']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    f1 = tf.keras.layers.Conv1D(filters=256, 
                                kernel_size=5, 
                                strides=1,
                                padding='causal',
                                activation='relu',
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros')
    f2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128,
                                                            activation='tanh',
                                                            recurrent_activation='sigmoid',
                                                            kernel_initializer='glorot_uniform',
                                                            bias_initializer='zeros',
                                                            recurrent_initializer='zeros',
                                                            return_sequences=True,
                                                            return_state = False), 
                                       merge_mode='concat')
    f3 = tf.keras.layers.Dropout(rate=0.5)
    f4 = tf.keras.layers.Dense(units=128,
                               activation='relu',
                               kernel_initializer='glorot_uniform',
                               bias_initializer='zeros')
    f5 = tf.keras.layers.Dropout(rate=0.5)
    f6 = tf.keras.layers.Dense(units=128,
                               activation='relu',
                               kernel_initializer='glorot_uniform',
                               bias_initializer='zeros')
    f7 = tf.keras.layers.Dropout(rate=0.5)
    f8 = tf.keras.layers.Dense(units=dims[1][1],
                                activation='linear',
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros')
    x = tf.keras.Input(shape=dims[0])
    a1 = f1(x)
    a2 = f2(a1)
    a3 = f3(a2)
    a4 = f4(a3)
    a5 = f5(a4)
    a6 = f6(a5)
    a7 = f7(a6)
    y = f8(a7)
    model = tf.keras.Model(x, y)   
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, 
                                         beta_1=0.9, 
                                         beta_2=0.999, 
                                         epsilon=1e-07)
    model.compile(loss='huber_loss', metrics=['mae'], optimizer=optimizer)
    model.summary()
    callback1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                                     patience=5,
                                                     min_delta=0.001,
                                                     factor=0.1, 
                                                     min_lr=0.0001)
    callback2 = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                 patience=20,
                                                 min_delta=0.001)
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=[callback1, callback2])
    return model

In [7]:
model = Model({'dims': [X_train.shape[1:], Y_train.shape[1:]], 
               'X_train': X_train,
               'Y_train': Y_train,
               'epochs': 100, 
               'learning_rate': 0.001,
               'batch_size': 64})

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 7)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 24, 256)           9216      
_________________________________________________________________
bidirectional (Bidirectional (None, 24, 256)           394240    
_________________________________________________________________
dropout (Dropout)            (None, 24, 256)           0         
_________________________________________________________________
dense (Dense)                (None, 24, 128)           32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 128)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 24, 128)           16512 

In [8]:
model.evaluate(X_train, Y_train)

69072/69072 [==============================] - 23s 339us/sample - loss: 0.0050 - mae: 0.0461


[0.005017461280838105, 0.04612022]

In [9]:
model.evaluate(X_test, Y_test)

17232/17232 [==============================] - 6s 332us/sample - loss: 0.0056 - mae: 0.0494


[0.005592083896390537, 0.049375907]